In [1]:
def parse(raw):
    return {i: r.replace('"', '') for i,r in [l.split(': ') for l in raw.splitlines()]}

In [2]:
rules_raw = """0: 4 1 5
1: 2 3 | 3 2
2: 4 4 | 5 5
3: 4 5 | 5 4
4: "a"
5: "b\""""
rules = parse(rules_raw)
rules

{'0': '4 1 5',
 '1': '2 3 | 3 2',
 '2': '4 4 | 5 5',
 '3': '4 5 | 5 4',
 '4': 'a',
 '5': 'b'}

In [319]:
rules_raw, data_raw = open('data/19.txt').read().split('\n\n')
rules = parse(rules_raw)
#rules

In [3]:
import re
from functools import cache
import ring
re_num = re.compile('\d+')

@cache
def expand(r: str):
    #print('new call', r)
    pos_dash = r.find('|')
    contains_num = re_num.findall(r)

    if pos_dash > 0:
        return expand(r[:pos_dash]).union(expand(r[pos_dash+1:]))

    if re_num.fullmatch(r.strip()):
        return expand(rules[contains_num[0]])

    if not contains_num and pos_dash < 0:
        return set([r.replace(' ', '')])    

    res = set()
    for m in re_num.finditer(r):
        #print(m)
        to_add = set(r[:m.span()[0]] + poss + r[m.span()[1]:] for poss in expand(m.group()))
        #print('to_add', to_add)
        expanded =  [expand(e.strip()) for e in to_add]
        res = res.union(*expanded)
    #print("res", res)

    return res

#print(expand('"a" "b"'))
all_rules = expand('0')


In [4]:
sum(1 for m in data_raw.splitlines() if m in all_rules)

NameError: name 'data_raw' is not defined

In [5]:
len(data_raw.splitlines())

NameError: name 'data_raw' is not defined

# Part 2

In [6]:
rules_raw, data_raw = """42: 9 14 | 10 1
9: 14 27 | 1 26
10: 23 14 | 28 1
1: "a"
11: 42 31
5: 1 14 | 15 1
19: 14 1 | 14 14
12: 24 14 | 19 1
16: 15 1 | 14 14
31: 14 17 | 1 13
6: 14 14 | 1 14
2: 1 24 | 14 4
0: 8 11
13: 14 3 | 1 12
15: 1 | 14
17: 14 2 | 1 7
23: 25 1 | 22 14
28: 16 1
4: 1 1
20: 14 14 | 1 15
3: 5 14 | 16 1
27: 1 6 | 14 18
14: "b"
21: 14 1 | 1 14
25: 1 1 | 1 14
22: 14 14
8: 42
26: 14 22 | 1 20
18: 15 15
7: 14 5 | 1 21
24: 14 1

abbbbbabbbaaaababbaabbbbabababbbabbbbbbabaaaa
bbabbbbaabaabba
babbbbaabbbbbabbbbbbaabaaabaaa
aaabbbbbbaaaabaababaabababbabaaabbababababaaa
bbbbbbbaaaabbbbaaabbabaaa
bbbababbbbaaaaaaaabbababaaababaabab
ababaaaaaabaaab
ababaaaaabbbaba
baabbaaaabbaaaababbaababb
abbbbabbbbaaaababbbbbbaaaababb
aaaaabbaabaaaaababaa
aaaabbaaaabbaaa
aaaabbaabbaaaaaaabbbabbbaaabbaabaaa
babaaabbbaaabaababbaabababaaab
aabbbbbaabbbaaaaaabbbbbababaaaaabbaaabba""".split('\n\n')
rules = parse(rules_raw)
expand.cache_clear()
#rules

In [7]:
all_rules = expand('0')
four2 = expand('42')
three1 = expand('31')

In [8]:
to_check = [m for m in data_raw.splitlines() if m not in all_rules]
len(to_check)

12

In [9]:
@cache
def get_31s(d):
    #print('call', d)
    p = set(d[len(c):] for c in three1 if d.startswith(c))
    if len(p) == 0:
        return set([d])
    for a in p:
        return set('31 ' + b for b in get_31s(a))
get_31s.cache_clear()

In [10]:
@cache
def get_42s(d):
    #print('call', d)
    p = set(d[len(c):] for c in four2 if d.startswith(c))
    if len(p) == 0:
        return set([d])
    for a in p:
        return set('42 ' + b for b in get_42s(a))
get_42s.cache_clear()
get_31s.cache_clear()

In [11]:
def is_valid(d: str):
    d = d.strip()
    s = set(d.split(' '))
    if not s in [set(['31', '42']), set(['42'])]:
        return False
    if not d.startswith('42'):
        return False
    if d.count('42 31') > 1:
        return False
    if d.count('31 42') > 0:
        return False
    if d.count('31') >= d.count('42'):
        return False
    return True
    


In [22]:
possibles = []
for c in to_check:
    res = set()
    for cand in get_42s(c):
        spl = cand.strip().split(' ')
        for replaced in get_31s(spl[-1]):
            res.add(" ".join(spl[:-1] + [replaced.strip()]))
        #res.update()
    
    possibles.append(res)

res = 0
for el in possibles:
    res += any(map(is_valid, el))
res


10